In [2]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV

In [138]:
df1 = pd.read_csv('E:\\MySQL_data\\ad.data', header = None, low_memory=False )
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,1549,1550,1551,1552,1553,1554,1555,1556,1557,1558
0,125,125,1.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
1,57,468,8.2105,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
2,33,230,6.9696,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
3,60,468,7.8,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
4,60,468,7.8,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.


In [139]:
explanatory_variable_columns = set( df1.columns.values )

print('explanatory_variable_columns的总列数为: ', len( explanatory_variable_columns ) )

explanatory_variable_columns.remove( len(df1.columns.values)-1 )
response_variable_columns = df1[1558]

print('df1的总行数为： ', len(df1) )

explanatory_variable_columns的总列数为:  1559
df1的总行数为：  3279


In [131]:
set(response_variable_columns)

{'ad.', 'nonad.'}

In [146]:
response_variable_columns = response_variable_columns[:]
response_variable_columns.replace({'ad.':int(1), 'nonad.':int(0)})
response_variable_columns.head()

0    1
1    1
2    1
3    1
4    1
Name: 1558, dtype: object

In [154]:
y = response_variable_columns.values.astype('int16')
# 一定要将y中的值设置为数字，否则grid_search.fit时就会出错。
print(set(y))
y[:10]

{0, 1}


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int16)

In [134]:
x = df1.iloc[:, :1558].copy() 

In [155]:
x.replace( to_replace=' *?', value=-1, regex=True, inplace=True )
x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.25 )

pipeline = Pipeline([
    ( 'clf', DecisionTreeClassifier(criterion='entropy') )
])
parameters = {
    'clf__max_depth':( 150, 155, 160 ),
    'clf__min_samples_split': (2, 3),
    'clf__min_samples_leaf': (1, 2, 3)
}
grid_search = GridSearchCV( pipeline, parameters, n_jobs=-1, verbose=1,
                          scoring='f1' )

In [145]:
grid_search.fit( x_train, y_train )

Fitting 3 folds for each of 18 candidates, totalling 54 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed:    7.3s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('clf', DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'clf__max_depth': (150, 155, 160), 'clf__min_samples_split': (2, 3), 'clf__min_samples_leaf': (1, 2, 3)},
       pre_dispatch='2*n_jobs', refit=True, scoring='f1', verbose=1)

In [151]:
best_parameters = grid_search.best_estimator_.get_params()
print('Best score:{:.3f}'.format(grid_search.best_score_ ) )
print('Best parameters set:')
for param_name in sorted(parameters.keys() ):
    print('{} :{}'.format( param_name, best_parameters[param_name] ))

predictions = grid_search.predict(x_test)
print(classification_report(y_test, predictions ))

Best score:0.832
Best parameters set:
clf__max_depth :150
clf__min_samples_leaf :1
clf__min_samples_split :3
             precision    recall  f1-score   support

          0       0.96      0.99      0.98       710
          1       0.94      0.76      0.84       110

avg / total       0.96      0.96      0.96       820

